# Training
- 찾아낸 best 하이퍼파라미터 조합으로 재튜닝

In [1]:
!pip install timm lpips torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.2 MB/s eta 0:00:00


In [2]:
#--Colab setup---
from google.colab import drive
import os
import sys
import shutil
# 1. 드라이브 마운트
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 프로젝트 경로 설정
PROJECT_DIR = "/content/drive/MyDrive/25_AML_OASIS_dataset"
if not os.path.exists(PROJECT_DIR):
    os.makedirs(PROJECT_DIR)
os.chdir(PROJECT_DIR)
print(f"작업 디렉토리: {os.getcwd()}")

# 3. 데이터 경로 로컬로 복사
DATA_DIR_SOURCE = os.path.join(PROJECT_DIR, "input")
DATA_DIR = "/content/input"

if not os.path.exists(DATA_DIR):
    shutil.copytree(DATA_DIR_SOURCE, DATA_DIR)

print(f" 이미지 데이터 경로: {DATA_DIR}")


Mounted at /content/drive
작업 디렉토리: /content/drive/MyDrive/25_AML_OASIS_dataset
 이미지 데이터 경로: /content/input


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from tqdm import tqdm
import numpy as np
import random
from glob import glob
from PIL import Image
import lpips
import timm
import traceback
import pandas as pd
import matplotlib.pyplot as plt
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import random
from scipy import linalg
from sklearn.linear_model import LogisticRegression
from huggingface_hub import hf_hub_download

# --- 설정 ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = 224
IMAGE_CHANNEL = 1
LATENT_DIM = 128
CLASSES = {0: 'Non Demented', 1: 'Very mild Dementia', 2: 'Mild Dementia'}
NUM_CLASSES = len(CLASSES)
CLASS_NAMES_MAP = {v: k for k, v in CLASSES.items()}
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])


# ==============================================================
# Dataset
# ==============================================================

class OASISFileDataset(Dataset):
    def __init__(self, root_dir, subject_list_file, transform=None):
        self.transform = transform
        self.data = []

        with open(subject_list_file, 'r') as f:
            valid_subjects = set(line.strip() for line in f.readlines())

        for class_folder in os.listdir(root_dir):
            if class_folder not in CLASS_NAMES_MAP:
              continue
            class_label = CLASS_NAMES_MAP[class_folder]


            for img_path in glob(os.path.join(root_dir, class_folder, "*.jpg")):
                name = os.path.basename(img_path).split('.')[0]
                parts = name.split('_')
                subj = parts[1] if len(parts) > 1 else name

                if subj in valid_subjects:
                    self.data.append((subj, img_path, class_label))

    def __len__(self):
      return len(self.data)

    def __getitem__(self, idx):
        subj, img_path, y = self.data[idx]
        img = Image.open(img_path).convert("L")
        if self.transform:
          img = self.transform(img)
        return img, y, subj

# ==============================================================
# Model Architecture
# ==============================================================


class FiLMCond(nn.Module):
    def __init__(self, embed_dim, num_features):
        super().__init__()
        self.scale = nn.Linear(embed_dim, num_features)
        self.shift = nn.Linear(embed_dim, num_features)

    def forward(self, h, class_emb):
        gamma = self.scale(class_emb).unsqueeze(-1).unsqueeze(-1)
        beta = self.shift(class_emb).unsqueeze(-1).unsqueeze(-1)
        return h * (1 + gamma) + beta

def up_block(cin, cout):
    return nn.Sequential(
        nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
        nn.Conv2d(cin, cout, 3, 1, 1),
        nn.BatchNorm2d(cout),
        nn.ReLU(inplace=True),
    )

class ImprovedCVAE(nn.Module):
    def __init__(self, latent_dim, img_size, img_channel, num_classes):
        super().__init__()
        self.latent_dim = latent_dim
        self.cls_dim = 32
        self.content_dim = latent_dim - self.cls_dim

        self.enc_conv1 = nn.Sequential(nn.Conv2d(img_channel, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU())
        self.enc_conv2 = nn.Sequential(nn.Conv2d(32, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU())
        self.enc_conv3 = nn.Sequential(nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU())
        self.enc_conv4 = nn.Sequential(nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.ReLU())

        self.flat = 256 * 14 * 14
        self.fc_mu = nn.Linear(self.flat, latent_dim)
        self.fc_log = nn.Linear(self.flat, latent_dim)

        self.class_embed = nn.Embedding(num_classes, self.cls_dim)

        self.film1 = FiLMCond(self.cls_dim, 128)
        self.film2 = FiLMCond(self.cls_dim, 64)
        self.film3 = FiLMCond(self.cls_dim, 32)

        self.dec_fc = nn.Linear(latent_dim, self.flat)
        self.dec1 = up_block(256 + 128, 128)
        self.dec2 = up_block(128 + 64, 64)
        self.dec3 = up_block(64, 32)
        self.out = nn.Sequential(nn.Conv2d(32, 1, 3, 1, 1), nn.Sigmoid())

    def reparam(self, mu, logv):
        std = (0.5 * logv).exp()
        return mu + torch.randn_like(std) * std

    def forward(self, x, class_label):
        e1 = self.enc_conv1(x)
        e2 = self.enc_conv2(e1)
        e3 = self.enc_conv3(e2)
        e4 = self.enc_conv4(e3)

        h = e4.view(x.size(0), -1)
        mu = self.fc_mu(h)
        logv = self.fc_log(h)
        z = self.reparam(mu, logv)

        z_content = z[:, :self.content_dim]
        z_class_raw = z[:, self.content_dim:]

        class_emb = self.class_embed(class_label)
        z_final = torch.cat([z_content, class_emb], dim=1)

        d = self.dec_fc(z_final).view(x.size(0), 256, 14, 14)
        d = F.interpolate(d, size=e3.shape[2:], mode="bilinear")
        d = self.dec1(torch.cat([d, e3], dim=1))
        d = self.film1(d, class_emb)

        d = F.interpolate(d, size=e2.shape[2:], mode="bilinear")
        d = self.dec2(torch.cat([d, e2], dim=1))
        d = self.film2(d, class_emb)

        d = F.interpolate(d, size=(112, 112), mode="bilinear")
        d = self.dec3(d)
        d = self.film3(d, class_emb)

        out = self.out(F.interpolate(d, size=(224, 224), mode="bilinear"))
        return out, mu, logv, z_content, z_class_raw

# ==============================================================
# Loss
# ==============================================================


# -- clustering loss --

class ImprovedClusteringLoss(nn.Module):
    def __init__(self, latent_dim, num_classes, margin=2.0):
        super().__init__()
        self.num_classes = num_classes
        self.margin = margin
        self.centers = nn.Parameter(torch.randn(num_classes, latent_dim))
        nn.init.xavier_uniform_(self.centers)

    def forward(self, features, labels):
        centers_batch = self.centers[labels]
        center_loss = F.mse_loss(features, centers_batch)

        separation_loss = 0
        num_pairs = 0
        for i in range(self.num_classes):
            for j in range(i + 1, self.num_classes):
                dist = F.pairwise_distance(self.centers[i].unsqueeze(0), self.centers[j].unsqueeze(0))
                separation_loss += F.relu(self.margin - dist)
                num_pairs += 1

        if num_pairs > 0:
            separation_loss /= num_pairs

        return center_loss, separation_loss



# --- classification loss: pretrained classifier ---

class SimpleClassifier(nn.Module):
    def __init__(self, model_name='resnet18', num_classes=NUM_CLASSES):
        super().__init__()
        self.model = timm.create_model(
            model_name,
            pretrained=False,
            num_classes=num_classes,
            in_chans=1)

    def forward(self, x):
        return self.model(x)

def load_classifier(path="best_classifier_resnet18_weights_42.pth"):

    cls = SimpleClassifier(num_classes=NUM_CLASSES).to(DEVICE)
    ckpt = torch.load(path, map_location=DEVICE)
    cls.load_state_dict(ckpt)
    cls.eval()
    for p in cls.parameters():
        p.requires_grad = False
    return cls



# --- LPIPS loss ---
lpips_fn = lpips.LPIPS(net='alex').to(DEVICE)
for p in lpips_fn.parameters():
    p.requires_grad = False


# ---------------------------------------------------------------
# Validation
# ---------------------------------------------------------------
@torch.no_grad()
def validate(model, loader, beta, clustering_loss_fn, classifier, config):
    model.eval()
    total = 0
    sum_loss = 0

    for x, y, _ in loader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        recon, mu, logv, z_content, z_class_raw = model(x, y)

        # loss
        bce = F.binary_cross_entropy(recon, x, reduction='mean')
        kld = -0.5 * torch.mean(1 + logv - mu.pow(2) - logv.exp())
        lp = lpips_fn(recon.repeat(1, 3, 1, 1), x.repeat(1, 3, 1, 1)).mean()
        center_loss, sep_loss = clustering_loss_fn(z_class_raw, y)

        # classifier loss
        recon_norm = (recon - 0.456) / 0.224
        cls_logits = classifier(recon_norm)
        cls_loss = F.cross_entropy(cls_logits, y)

        # total loss
        loss = (config['lambda_recon'] * bce +
                beta * kld +
                config['lambda_lpips'] * lp +
                config['w_center'] * center_loss +
                config['w_sep'] * sep_loss +        # W_SEPARATION -> config['w_sep']
                config['lambda_cls'] * cls_loss)    # CLASSIFIER_LOSS_WEIGHT -> config['lambda_cls']

        bs = x.size(0)
        total += bs
        sum_loss += loss.item() * bs

    return sum_loss / total

# ---------------------------------------------------------------
# Training
# ---------------------------------------------------------------
def train_improved_cvae(model, train_loader, val_loader, classifier, config):

    epochs = config['epochs']
    lr = config['lr']
    beta_max = config['beta']

    # 저장 경로
    save_path = f"exp_{config['exp_name']}.pth"

    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=epochs)

    clustering_loss_fn = ImprovedClusteringLoss(
        latent_dim=model.cls_dim,
        num_classes=NUM_CLASSES,
        margin=config['margin']
    ).to(DEVICE)

    best_val = 1e9
    patience = 10
    no_improve = 0


    for epoch in range(1, epochs + 1):
        model.train()

        # KL Annealing
        beta = min(beta_max, beta_max * epoch / max(1, epochs // 2))

        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}", leave=False)
        for x, y, _ in pbar:
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            opt.zero_grad()
            recon, mu, logv, z_content, z_class_raw = model(x, y)

            bce = F.binary_cross_entropy(recon, x, reduction='mean')
            kld = -0.5 * torch.mean(1 + logv - mu.pow(2) - logv.exp())
            lp = lpips_fn(recon.repeat(1, 3, 1, 1), x.repeat(1, 3, 1, 1)).mean()
            center_loss, sep_loss = clustering_loss_fn(z_class_raw, y)

            recon_norm = (recon - 0.456) / 0.224
            cls_logits = classifier(recon_norm)
            cls_loss = F.cross_entropy(cls_logits, y)


            loss = (config['lambda_recon'] * bce +
                    beta * kld +
                    config['lambda_lpips'] * lp +
                    config['w_center'] * center_loss +
                    config['w_sep'] * sep_loss +
                    config['lambda_cls'] * cls_loss)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()

            pbar.set_postfix({
                "loss": f"{loss.item():.3f}",
                "cls": f"{cls_loss.item():.3f}"
            })

        scheduler.step()


        val_loss = validate(
            model, val_loader, beta,
            clustering_loss_fn, classifier, config
        )

        print(f"Ep {epoch}/{epochs} | Val Loss {val_loss:.4f}")

        if val_loss < best_val:
            best_val = val_loss
            torch.save({
                "model": model.state_dict(),
                "cluster": clustering_loss_fn.state_dict(),
                "epoch": epoch,
                "val_loss": val_loss,
                "hparams": config
            }, save_path)
            print(f"Saved best: {save_path}\n")
            no_improve = 0

        else:
            no_improve += 1
            if no_improve >= patience:
                print("Early stopping")
                break


    return clustering_loss_fn

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 191MB/s]


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:

if __name__ == "__main__":

    # (1)데이터셋 로드
    train_dataset = OASISFileDataset(DATA_DIR, "train_subjects.txt", transform=transform)
    val_dataset = OASISFileDataset(DATA_DIR, "val_subjects.txt", transform=transform)

    # (2) 실험 설정
    base_config = {
        'epochs': 20,           # 실험용 에폭
        'batch_size': 32,
        'lr': 1e-4,
        'lambda_recon': 1.0,
        'beta': 2.0,
        'lambda_lpips': 1.0,
        'lambda_cls': 2.0,
        'w_center': 10.0,
        'w_sep': 5.0,
        'margin': 2.0

    }

    train_loader = DataLoader(train_dataset, batch_size=base_config['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=base_config['batch_size'], shuffle=False, num_workers=2)

    # Classifier 로드
    classifier = load_classifier()



    experiments_list = [

        # best1
        {'exp_name': 'best1_lpips1.5_wsep0_mar1.8',  'lambda_lpips': 1.5, 'w_sep':0.0, 'margin':1.8},
        # best 2
        {'exp_name': 'best2_beta1_lpips2_wcen5_wsep0_cls1',  'beta': 1.0, 'lambda_lpips':2.0, 'w_center':5.0, 'w_sep':0.0, 'lambda_cls':1.0},
        # best 3
        {'exp_name': 'best3_wsep0_cls2.5_mar1.8', 'lambda_cls': 2.5, 'w_sep':0.0, 'margin':1.8},

    ]

    # (3) 루프 실행


    for i, exp_params in enumerate(experiments_list):
        print(f"[{i+1}/{len(experiments_list)}] for {exp_params['exp_name']}...")

        config = base_config.copy()
        config.update(exp_params)

        model = ImprovedCVAE(
            LATENT_DIM, IMAGE_SIZE, IMAGE_CHANNEL, num_classes=NUM_CLASSES
        ).to(DEVICE)

        try:
            train_improved_cvae(model, train_loader, val_loader, classifier, config)
        except Exception as e:
            print(f"Error in experiment {config['exp_name']}: {e}")

    print("\n모든 실험이 완료")


🏁 총 3개의 실험을 순차적으로 시작합니다.

▶ [1/3] Setting up best1_lpips1.5_wsep0_mar1.8...


Ep 1/20 | Val Loss 2.6937
Saved best: exp_best1_lpips1.5_wsep0_mar1.8.pth



Ep 2/20 | Val Loss 2.4170
Saved best: exp_best1_lpips1.5_wsep0_mar1.8.pth



Ep 3/20 | Val Loss 1.3185
Saved best: exp_best1_lpips1.5_wsep0_mar1.8.pth



Ep 4/20 | Val Loss 1.3125
Saved best: exp_best1_lpips1.5_wsep0_mar1.8.pth



...

Ep 10/20 | Val Loss 1.6481


Ep 11/20 | Val Loss 1.6392


Ep 12/20 | Val Loss 1.6239


Ep 13/20 | Val Loss 1.6364


Ep 14/20 | Val Loss 1.6211
Early stopping

모든 실험이 완료


# Evaluation

## Eval_1

In [4]:
# ==============================================================
# 설정: ckpt 파일 로컬로 복사
# ==============================================================
import shutil
from glob import glob

DRIVE_CKPT_DIR = "/content/drive/MyDrive/25_AML_OASIS_dataset" # 드라이브 경로
LOCAL_CKPT_DIR = "/content/checkpoints"                        # 로컬 임시 경로

os.makedirs(LOCAL_CKPT_DIR, exist_ok=True)

ckpt_files = glob(os.path.join(DRIVE_CKPT_DIR, "exp_best*.pth"))
print(f"{len(ckpt_files)}개 ckpt 파일")

# 3. 복사 실행
for f in ckpt_files:
    file_name = os.path.basename(f)
    dest_path = os.path.join(LOCAL_CKPT_DIR, file_name)

    if not os.path.exists(dest_path):
        shutil.copy(f, LOCAL_CKPT_DIR)
        print(f"copied: {file_name}")
    else:
        print(f"skipped (exists): {file_name}")


3개 ckpt 파일
copied: exp_best1_lpips1.5_wsep0_mar1.8.pth
copied: exp_best2_beta1_lpips2_wcen5_wsep0_cls1.pth
copied: exp_best3_wsep0_cls2.5_mar1.8.pth


In [5]:
# ==============================================================
# 시각화 및 저장 함수
# ==============================================================
@torch.no_grad()
def save_comparison_plot(model, image, orig_label, target_label, save_path):
    model.eval()
    img = image.to(DEVICE).unsqueeze(0)
    target = torch.tensor([target_label], device=DEVICE)
    recon, _, _, _, _ = model(img, target)

    x0_np = image.squeeze().cpu().numpy()
    gen_np = recon.squeeze().cpu().clamp(0, 1).numpy()

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(x0_np, cmap='gray')
    plt.title(f"Orig ({CLASSES.get(orig_label.item())})")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(gen_np, cmap='gray')
    plt.title(f"Gen ({CLASSES.get(target_label.item())})")
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()



In [6]:
# ==============================================================
# Evaluation 1: Image Quality Metrics
# ==============================================================

def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse)).item()

def calculate_laplacian_sharpness(images):
    kernel = torch.tensor([[0, 1, 0], [1, -4, 1], [0, 1, 0]],
                         dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(images.device)
    edge_map = F.conv2d(images, kernel, padding=1)
    var = torch.var(edge_map.view(images.size(0), -1), dim=1)
    return var.mean().item()

def calculate_js_divergence(real_imgs, fake_imgs, bins=256):
    real_pixels = (real_imgs.flatten() * 255).long()
    fake_pixels = (fake_imgs.flatten() * 255).long()

    hist_real = torch.histc(real_pixels.float(), bins=bins, min=0, max=255)
    hist_fake = torch.histc(fake_pixels.float(), bins=bins, min=0, max=255)

    p = hist_real / (hist_real.sum() + 1e-6)
    q = hist_fake / (hist_fake.sum() + 1e-6)
    m = 0.5 * (p + q)

    kl_pm = F.kl_div((m + 1e-10).log(), p, reduction='batchmean')
    kl_qm = F.kl_div((m + 1e-10).log(), q, reduction='batchmean')

    js_div = 0.5 * kl_pm + 0.5 * kl_qm
    return js_div.item()




In [7]:
# load pretrained classifier for evaluation

class PretrainedClassifier(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True):
        super().__init__()
        # in_chans=1 설정으로 1채널 입력 가능하게 함
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes, in_chans=1)

    def forward(self, x):
        return self.model(x)


def load_eval_classifier(path):

    model = PretrainedClassifier(model_name='densenet121', num_classes=NUM_CLASSES, pretrained=False)
    model = model.to(DEVICE)

    try:
        ckpt = torch.load(path, map_location=DEVICE)
        model.load_state_dict(ckpt)
        print("classifier ok")
    except Exception as e:
        print(f"failed: {e}")
        return None

    model.eval()
    for p in model.parameters():
        p.requires_grad = False

    return model


In [8]:
def evaluate_all_checkpoints(
    ckpt_dir,
    val_list_path,
    classifier_path,
    data_root,
    output_csv='final_evaluation_results.csv',
    val_subset=0  # 0: 전체, >0: 서브셋
):


    lpips_metric = LearnedPerceptualImagePatchSimilarity(net_type='vgg').to(DEVICE)
    ssim_metric = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0).to(DEVICE)
    classifier = load_eval_classifier(classifier_path)

    val_dataset = OASISFileDataset(data_root, val_list_path, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    ckpt_files = sorted(glob(os.path.join(ckpt_dir, "exp_*.pth")))
    results = []


    for ckpt_path in tqdm(ckpt_files, desc="Evaluating"):
      checkpoint = torch.load(ckpt_path, map_location=DEVICE)
      exp_id = os.path.basename(ckpt_path).replace(".pth", "")

      model = ImprovedCVAE(LATENT_DIM, IMAGE_SIZE, IMAGE_CHANNEL, NUM_CLASSES).to(DEVICE)
      model.load_state_dict(checkpoint['model'])
      model.eval()

      cluster_loss_fn = ImprovedClusteringLoss(model.cls_dim, NUM_CLASSES).to(DEVICE)
      if 'cluster' in checkpoint:
          cluster_loss_fn.load_state_dict(checkpoint['cluster'])

      hparams = checkpoint.get('hparams', {})

      total_lpips = total_ssim = total_conf = total_acc = 0.0
      total_lap = total_js = total_psnr = 0.0
      n_samples = 0

      z_collections = {c: [] for c in range(NUM_CLASSES)}

      # for saving image & excels
      exp_id = os.path.basename(ckpt_path).replace(".pth", "")
      epoch = checkpoint.get('epoch', -1)
      vis_save_dir = os.path.join("eval_outputs", exp_id)
      os.makedirs(vis_save_dir, exist_ok=True)

      with torch.no_grad():
            for imgs, labels, _ in val_loader:
                imgs = imgs.to(DEVICE)
                labels = labels.to(DEVICE)
                target_labels = (labels + 1) % NUM_CLASSES

                recon, _, _, _, z_class_raw = model(imgs, target_labels)
                recon_self, _, _, _, _ = model(imgs, labels) #for PSNR evaluation

                recon_clipped = recon.clamp(0, 1)
                recon_self_clipped = recon_self.clamp(0, 1)

                # Metrics: regarding classification
                if classifier:
                    recon_norm = (recon - 0.456) / 0.224
                    logits = classifier(recon_norm)
                    probs = torch.softmax(logits, dim=1)

                    # confidence score
                    target_probs = probs.gather(1, target_labels.view(-1, 1))
                    total_conf += target_probs.sum().item()

                    # accuracy
                    preds = torch.argmax(probs, dim=1)
                    total_acc += (preds == target_labels).sum().item()

                # Metrics: LPIPS & SSIM
                imgs_3c = imgs.repeat(1, 3, 1, 1)
                recon_3c = recon_clipped.repeat(1, 3, 1, 1)

                batch_lpips = lpips_metric(recon_3c, imgs_3c)
                total_lpips += batch_lpips.item() * imgs.size(0)

                batch_ssim = ssim_metric(recon_clipped, imgs)
                total_ssim += batch_ssim.item() * imgs.size(0)

                # Metrics: Laplacian, JS divergence, PSNR
                total_lap += calculate_laplacian_sharpness(recon_clipped) * imgs.size(0)
                total_js += calculate_js_divergence(imgs, recon_clipped) * imgs.size(0)
                total_psnr += calculate_psnr(recon_self_clipped, imgs) * imgs.size(0)


                for z, y in zip(z_class_raw, target_labels):
                    z_collections[y.item()].append(z.cpu().numpy())

                n_samples += imgs.size(0)

                saved_count = 0
                # Save images (Top 5 per model)
                if saved_count < 5:
                    for i in range(imgs.size(0)):
                        if saved_count >= 5: break
                        fname = f"sample_{saved_count}.png"
                        save_comparison_plot(model, imgs[i], labels[i], target_labels[i],
                                              os.path.join(vis_save_dir, fname))
                        saved_count += 1

            avg_lpips = total_lpips / n_samples
            avg_ssim = total_ssim / n_samples
            avg_conf = total_conf / n_samples if classifier else 0.0
            avg_acc = total_acc / n_samples if classifier else 0.0
            avg_psnr = total_psnr/ n_samples
            avg_lap = total_lap / n_samples
            avg_js = total_js / n_samples



            # Compute Latent Metrics
            intra_vars = []
            for c in z_collections:
                if len(z_collections[c]) > 1:
                    arr = np.stack(z_collections[c])
                    # Feature 차원(32)에 대한 분산의 평균
                    intra_vars.append(np.mean(np.var(arr, axis=0)))
            mean_intra_var = np.mean(intra_vars) if intra_vars else 0.0

            centers = cluster_loss_fn.centers.detach()
            centers = F.normalize(centers, dim=1)
            dist_matrix = torch.cdist(centers, centers, p=2)
            mask = ~torch.eye(NUM_CLASSES, dtype=bool, device=DEVICE)
            mean_dist = dist_matrix[mask].mean().item()

            # Result Dict
            row = {
                "exp_id": exp_id,
                "Target_Acc": avg_acc,
                "Target_Conf": avg_conf,
                "LPIPS": avg_lpips,
                "MS_SSIM": avg_ssim,
                "Intra_Var": mean_intra_var,
                "Inter_Dist": mean_dist,
                "Laplacian_Sharpness": avg_lap,
                "JS_Divergence": avg_js,
                "PSNR_Score":avg_psnr,
                **hparams
            }
            results.append(row)


    # Save CSV
    df = pd.DataFrame(results)


    priority = [
        'exp_id',
        'Target_Acc', 'Target_Conf',
        'LPIPS', 'MS_SSIM',
        'Laplacian_Sharpness', 'JS_Divergence',
        'Intra_Var', 'Inter_Dist', 'PSNR_Score'
    ]
    others = [c for c in df.columns if c not in priority]
    df = df[priority + others]

    df.to_csv(output_csv, index=False)
    return df

In [12]:
if __name__ == "__main__":
    print("[Evaluation Only] Starting full validation...")

    evaluate_all_checkpoints(
        ckpt_dir= LOCAL_CKPT_DIR,
        val_list_path="val_subjects.txt",
        classifier_path="best_classifier_densenet121_weights_42.pth",
        data_root=DATA_DIR,
        output_csv="best_evaluation_results_1201.csv",
        val_subset=0
    )

[Evaluation Only] Starting full validation...
classifier ok


Evaluating: 100%|██████████| 3/3 [04:09<00:00, 83.19s/it]


## Eval 2

In [9]:
# =========================================================================
# [Evaluation 2] Final Check: FMD, KID, C2ST(5-Fold)
# =========================================================================

# --- feature extractor ---
class MedicalFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        try:
            # RadImageNet 로드 시도
            model_path = hf_hub_download(repo_id="Lab-Rasool/RadImageNet", filename="ResNet50.pt")
            loaded_object = torch.load(model_path, map_location='cpu')
            if isinstance(loaded_object, nn.Module): state_dict = loaded_object.state_dict()
            else: state_dict = loaded_object

            self.backbone = timm.create_model('resnet50', pretrained=False, num_classes=0)
            self.backbone.load_state_dict(state_dict, strict=False)
            print("ok")
        except:
            print("Nope-> ImageNet")
            self.backbone = timm.create_model('resnet50', pretrained=True, num_classes=0)

        self.backbone.eval()
        for p in self.backbone.parameters(): p.requires_grad = False

    def forward(self, x):
        if x.shape[1] == 1: x = x.repeat(1, 3, 1, 1)
        return self.backbone(x)

# --- Metrics ---
def calculate_fmd(feats1, feats2, eps=1e-6):
    mu1, sigma1 = np.mean(feats1, axis=0), np.cov(feats1, rowvar=False)
    mu2, sigma2 = np.mean(feats2, axis=0), np.cov(feats2, rowvar=False)
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
    if np.iscomplexobj(covmean): covmean = covmean.real
    return float(diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean))

def calculate_kid(feats1, feats2, degree=3, gamma=None, coef0=1.0):
    def polynomial_kernel(X, Y):
        if gamma is None: _gamma = 1.0 / X.shape[1]
        else: _gamma = gamma
        return (np.dot(X, Y.T) * _gamma + coef0) ** degree
    m, n = feats1.shape[0], feats2.shape[0]
    K_xx = polynomial_kernel(feats1, feats1)
    K_yy = polynomial_kernel(feats2, feats2)
    K_xy = polynomial_kernel(feats1, feats2)
    mmd2 = (np.sum(K_xx)-np.trace(K_xx))/(m*(m-1)) + (np.sum(K_yy)-np.trace(K_yy))/(n*(n-1)) - 2*np.sum(K_xy)/(m*n)
    return float(mmd2)

def calculate_c2st_5fold(feats_real, feats_fake):
    """
    Real(0) vs Fake(1) 분류 정확도를 5-Fold 교차 검증으로 계산.
    0.5에 가까울수록 좋음 (구분 불가).
    """
    X = np.concatenate([feats_real, feats_fake], axis=0)
    y = np.concatenate([np.zeros(len(feats_real)), np.ones(len(feats_fake))], axis=0)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    # Logistic Regression (Fast)
    clf = LogisticRegression(max_iter=1000, solver='lbfgs', n_jobs=-1)

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        clf.fit(X_train, y_train)
        acc = accuracy_score(y_test, clf.predict(X_test))
        scores.append(acc)

    return np.mean(scores), np.std(scores)


# IC-LPIPS (Medical Feature Cosine Distance)
def calculate_iclpips(feats_real, feats_fake):
    """
    RadImageNet 특징 간의 코사인 거리를 계산합니다. (낮을수록 유사함)
    뇌 구조 보존을 측정
    """

    norm_real = np.linalg.norm(feats_real, axis=1, keepdims=True)
    norm_fake = np.linalg.norm(feats_fake, axis=1, keepdims=True)

    norm_real = np.where(norm_real == 0, 1e-10, norm_real)
    norm_fake = np.where(norm_fake == 0, 1e-10, norm_fake)

    feats_real_norm = feats_real / norm_real
    feats_fake_norm = feats_fake / norm_fake

    similarity = np.sum(feats_real_norm * feats_fake_norm, axis=1)

    distance = 1 - similarity

    return float(np.mean(distance))

def run_evaluation_2(ckpt_dir, data_root, val_list_path, output_csv="best_results_eval2_1201.csv"):


    extractor = MedicalFeatureExtractor().to(DEVICE)
    ds_val = OASISFileDataset(data_root, val_list_path, transform=transform)
    # 배치 128로 빠르게 추출
    loader_val = DataLoader(ds_val, batch_size=128, shuffle=False, num_workers=2)


    real_feats = []
    with torch.no_grad():
        for x, _, _ in tqdm(loader_val, desc="Real Feats"):
            real_feats.append(extractor(x.to(DEVICE)).cpu().numpy())
    real_feats = np.concatenate(real_feats, axis=0)


    ckpt_files = sorted(glob(os.path.join(ckpt_dir, "exp_best*.pth")))
    results = []

    pbar = tqdm(ckpt_files, desc="Models", unit="ckpt")
    for ckpt_path in pbar:
        try:
            exp_id = os.path.basename(ckpt_path).replace(".pth", "")
            pbar.set_description(f"Eval: {exp_id}")


            ckpt = torch.load(ckpt_path, map_location=DEVICE)
            hparams = ckpt.get('hparams', {})
            model = ImprovedCVAE(LATENT_DIM, IMAGE_SIZE, IMAGE_CHANNEL, NUM_CLASSES).to(DEVICE)
            model.load_state_dict(ckpt['model'])
            model.eval()


            fake_feats = []
            with torch.no_grad():
                for x, y, _ in loader_val:
                    x = x.to(DEVICE); y = y.to(DEVICE)
                    target_y = (y + 1) % NUM_CLASSES
                    recon, _, _, _, _ = model(x, target_y)
                    fake_feats.append(extractor(recon).cpu().numpy())
            fake_feats = np.concatenate(fake_feats, axis=0)

            # Metrics
            fmd = calculate_fmd(real_feats, fake_feats)
            kid = calculate_kid(real_feats, fake_feats)
            c2st_mean, c2st_std = calculate_c2st_5fold(real_feats, fake_feats)
            iclpips = calculate_iclpips(real_feats, fake_feats)

            row = {
                "exp_id": exp_id,
                "FMD": fmd,
                "KID": kid,
                "C2ST_Mean": c2st_mean,
                "C2ST_Std": c2st_std,
                "ICLPIPS": iclpips,
                **hparams
            }
            results.append(row)
            print(f"   -> {exp_id} | FMD={fmd:.2f} | C2ST={c2st_mean:.3f}(±{c2st_std:.3f})")

        except Exception as e:
            print(f"error {e}")

    # Save
    df = pd.DataFrame(results)
    cols = ['exp_id', 'FMD', 'KID', 'ICLPIPS','C2ST_Mean', 'C2ST_Std'] + [c for c in df.columns if c not in ['exp_id', 'FMD', 'KID', 'C2ST_Mean', 'C2ST_Std']]
    df = df[cols]
    df.to_csv(output_csv, index=False)
    print(output_csv)
    return df

# 실행
if __name__ == "__main__":
    run_evaluation_2(
        ckpt_dir=".",
        data_root="/content/input",
        val_list_path="val_subjects.txt"
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ResNet50.pt:   0%|          | 0.00/94.3M [00:00<?, ?B/s]

ok


Eval: exp_best1_lpips1.5_wsep0_mar1.8:   0%|          | 0/3 [00:00<?, ?ckpt/s]/tmp/ipython-input-3507283142.py:35: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
/tmp/ipython-input-3507283142.py:35: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
Eval: exp_best2_beta1_lpips2_wcen5_wsep0_cls1:  33%|███▎      | 1/3 [00:33<01:07, 33.75s/ckpt]

   -> exp_best1_lpips1.5_wsep0_mar1.8 | FMD=0.00 | C2ST=0.597(±0.013)


/tmp/ipython-input-3507283142.py:35: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
/tmp/ipython-input-3507283142.py:35: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
Eval: exp_best3_wsep0_cls2.5_mar1.8:  67%|██████▋   | 2/3 [01:05<00:32, 32.43s/ckpt]          

   -> exp_best2_beta1_lpips2_wcen5_wsep0_cls1 | FMD=0.00 | C2ST=0.553(±0.023)


/tmp/ipython-input-3507283142.py:35: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
/tmp/ipython-input-3507283142.py:35: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
Eval: exp_best3_wsep0_cls2.5_mar1.8: 100%|██████████| 3/3 [01:34<00:00, 31.52s/ckpt]

   -> exp_best3_wsep0_cls2.5_mar1.8 | FMD=0.02 | C2ST=0.709(±0.008)


best_results_eval2_1201.csv
